In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

ticks = ["DPZ", "AAPL", "GOOG", "AMD", "GME", "SPY", "NFLX", "BA", "WMT","TWTR","GS","XOM","NKE","FEYE", "FB","BRK-B", "MSFT"]
for j in range (0, len(ticks)-1):
    company = ticks[j]
    start = dt.datetime(2012,1,1)
    end = dt.datetime(2020,1,1)
    data= web.DataReader(company, 'yahoo', start,end)
    scaler=MinMaxScaler(feature_range=(0,1))
    scaled_data=scaler.fit_transform(data['Close'].values.reshape(-1,1))
    prediction_days=60
    x_train=[]
    y_train=[]
    for x in range(prediction_days,len(scaled_data)):
        x_train.append(scaled_data[x-prediction_days:x,0])
        y_train.append(scaled_data[x,0])
    
    x_train, y_train=np.array(x_train), np.array(y_train)
    x_train=np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    
    model=Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1)) #Prediction

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=25, batch_size=32)
    test_start=dt.datetime(2020,1,1)
    test_end=dt.datetime.now()
    test_data = web.DataReader(company, 'yahoo', test_start, test_end)
    actual_prices = test_data['Close'].values
    total_dataset=pd.concat((data['Close'], test_data['Close']))
    model_inputs=total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
    model_inputs=model_inputs.reshape(-1,1)
    model_inputs=scaler.transform(model_inputs)
    
    x_test=[]
    for x in range(prediction_days, len(model_inputs)):
        x_test.append(model_inputs[x-prediction_days:x,0])
    
    x_test=np.array(x_test)
    x_test=np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
 
    predicted_prices= model.predict(x_test)
    predicted_prices= scaler.inverse_transform(predicted_prices)
    
    plt.plot(actual_prices, color="black", label=f"Actual {company} Price")
    plt.plot(predicted_prices, color="green", label=f"Predicted {company} Price")
    plt.title("Share Price")
    plt.xlabel('Time')
    plt.ylabel("Share Price")
    plt.legend()
    plt.show()
    
    buy=[]
    sell=[]

    for i in range (1, len(actual_prices)-1):
        if predicted_prices[i]-actual_prices[i]>0 and predicted_prices[i-1]-actual_prices[i-1]<=0:
            buy.append(i)
        if predicted_prices[i]-actual_prices[i]<0 and predicted_prices[i-1]-actual_prices[i-1]>=0:
            sell.append(i)
        
    print(buy)
    print(sell)